<a href="https://colab.research.google.com/github/swastika49/swastika49/blob/main/EfficientNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

print("Path to dataset files:", path)

100%|██████████| 736M/736M [00:05<00:00, 136MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="/root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1/dataset")

Resolving data files:   0%|          | 0/4217 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'glaucoma'

In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("google/efficientnet-b0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [ ]:
def preprocess_images(example):
    inputs = processor(example['image'], return_tensors="pt")
    example['pixel_values'] = inputs['pixel_values'].squeeze()  # Remove extra batch dimension
    return example

In [ ]:
processed_dataset = dataset.map(preprocess_images)

Map:   0%|          | 0/4217 [00:00<?, ? examples/s]

In [ ]:
splits = processed_dataset["train"].train_test_split(test_size=0.3)
val_test_split = splits['test'].train_test_split(test_size=0.5)  # 15% validation, 15% test

train_ds = splits['train']
val_ds = val_test_split['train']
test_ds = val_test_split['test']

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    "google/efficientnet-b0",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

config.json:   0%|          | 0.00/70.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([4, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Mediscan'/EfficientNet",
    per_device_train_batch_size=16,
    eval_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=200,
    eval_steps=200,
    logging_steps=100,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np

from evaluate import load
# Collate function for creating batches
def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Define the accuracy metric
metric = load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [14]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()
print("Train Results:", train_results)

Step,Training Loss,Validation Loss,Accuracy
200,0.330300,0.330328,0.902054


Step,Training Loss,Validation Loss,Accuracy
200,0.330300,0.330328,0.902054
400,0.173200,0.816279,0.905213
600,0.107200,0.261778,0.925750


***** train metrics *****
  epoch                    =        4.0
  total_flos               = 39841119GF
  train_loss               =     0.2255
  train_runtime            = 1:13:45.36
  train_samples_per_second =      2.667
  train_steps_per_second   =      0.167
Train Results: TrainOutput(global_step=740, training_loss=0.22546956216966785, metrics={'train_runtime': 4425.3672, 'train_samples_per_second': 2.667, 'train_steps_per_second': 0.167, 'total_flos': 4.277907641755238e+16, 'train_loss': 0.22546956216966785, 'epoch': 4.0})


In [15]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print("Evaluate Results:", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9258
  eval_loss               =     0.2618
  eval_runtime            = 0:02:11.70
  eval_samples_per_second =      4.806
  eval_steps_per_second   =      0.607
Evaluate Results: {'eval_loss': 0.26177799701690674, 'eval_accuracy': 0.9257503949447078, 'eval_runtime': 131.7023, 'eval_samples_per_second': 4.806, 'eval_steps_per_second': 0.607, 'epoch': 4.0}


In [16]:
test_results = trainer.evaluate(test_ds)
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.28139621019363403, 'eval_accuracy': 0.9052132701421801, 'eval_runtime': 122.4095, 'eval_samples_per_second': 5.171, 'eval_steps_per_second': 0.654, 'epoch': 4.0}
